In [ ]:
team_id = {}
with open("team_id.csv", "r") as f:
    for line in f.readlines()[1:]:
        team, id = line.split(",")
        team_id[int(id.strip())] = team

In [ ]:
#Function to add rolling average columns to dataframe

def rolling_avgs(df, cols, new_cols):
    for col, new_col in zip(cols, new_cols):
        df[new_col + "_3"] = df[col].rolling(3, closed="left").mean()
        df["cumsum"] = df[col].cumsum()
        df["index_val"] = range(1, len(df) + 1)
        df[new_col + "_total"] = df["cumsum"].shift(1).fillna(0) / df["index_val"]
        df.drop(["cumsum", "index_val"], axis=1, inplace=True)

In [ ]:
cols = ["kills/set", "errors/set", "total_attacks/set", "hit_pct", "assists/set", "aces/set", "serr/set", "digs/set", "b_solo/set", "b_assist/set", "b_error/set", "pts/set"]
new_cols = [f"rolling_{c}" for c in cols]

for root, dirs, filenames in os.walk("all_schedules"):
    for filename in fnmatch.filter(filenames, "*.csv"):
        df = pd.read_csv(os.path.join(root, filename), delimiter=",")
        rolling_avgs(df, cols, new_cols)
        df.to_csv(os.path.join(root, filename), index=False)

In [ ]:
bad = []
for key, l in games.items():
    if len(l) != 2:
        bad.append(key)

In [ ]:
# Function to separate opponent and venue

def seperate_opp_venue(x):
    opp_ven = x["opponent/venue"]
    res = ""
    if "@" not in opp_ven:
        res = opp_ven.strip()
    elif opp_ven.startswith("@"):
        res =  opp_ven.split("@")[-1].strip()
    else:
        res = opp_ven.split("@")[0].strip()
    return res.split("(")[0]

In [ ]:
# Some set data was impossible/incorrect, extract sets from result seems reliable
def sets_from_result(x):
    return sum([int(val.strip()) for val in x["result"].split("-")])

In [ ]:
cols = ["kills", "errors", "total_attacks", "assists", "aces", "serr", "digs", "b_solo", "b_assist", "b_error", "pts"]
new_cols = [f"{c}/set" for c in cols]


for root, dirs, filenames in os.walk("all_schedules"):
    for filename in fnmatch.filter(filenames, "*.csv"):
        df = pd.read_csv(os.path.join(root, filename), delimiter=",")
        df["sets_from_result"] = df.apply(lambda x: sum([int(val.strip()) for val in x["result"].split("-")]), 1)
        for col, new_col in zip(cols, new_cols):
            df[new_col] = df.apply(lambda x: x[col] / x["sets_from_result"], 1)
        df.to_csv(os.path.join(root, filename), index=False)